In [ ]:
!pip install -q pysndfx SoundFile audiomentations pretrainedmodels efficientnet_pytorch resnest

# No Random Sampling, only first 5 seconds.

In [ ]:
%%capture
!pip install colorednoise
import colorednoise as cn
import numpy as np
import librosa as lb
import torchvision
import warnings
warnings.filterwarnings('ignore')

import librosa.display as lbd
import soundfile as sf
from  soundfile import SoundFile
import pandas as pd
from  IPython.display import Audio
from pathlib import Path

import glob
import torch
from torch import nn, optim
import torch.nn.functional as F
from  torch.utils.data import Dataset, DataLoader
import torchvision

from resnest.torch import resnest50

from matplotlib import pyplot as plt

import os, random, gc
import re, time, json
from  ast import literal_eval


from IPython.display import Audio
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.model_selection import StratifiedKFold

from tqdm.notebook import tqdm
import joblib
import pytorch_lightning as pl

from efficientnet_pytorch import EfficientNet
import pretrainedmodels
from resnest.torch import resnest50_fast_1s1x64d
from resnest.torch import resnest50

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()
from fastai.vision.all import *

# Config Vars

In [ ]:
NUM_CLASSES = 397
SR = 32_000
DURATION = 20
MAX_READ_SAMPLES = 1 
audio_image_store = None
DATA_ROOT = Path("../input/birdclef-2021")
import glob

MEL_PATHS = sorted([Path(x) for x in glob.glob('../input/*birdclef-mels-computer-public*/rich_train_metadata.csv')] + [Path(x) for x in glob.glob("../input/k/andrewshao2005/*birdclef-*/rich_train_metadata.csv")])
TRAIN_LABEL_PATHS = sorted([Path(x) for x in glob.glob("../input/*birdclef-mels-computer-public*/LABEL_IDS.json")] +  [Path(x) for x in glob.glob("../input/k/andrewshao2005/*birdclef-*/LABEL_IDS.json")])
SOUNDSCAPES_PATH = '../input/birdclef-2021/train_soundscapes/'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ResNestPath = '../input/birds-cp-1/resnest50_fast_1s1x64d_conf_0.pt'
ResNextPath = '../input/birds-cp-2/resnext50_32x4d__0.pt'

external_soundscapes = '../input/external-xenocanto-data-soundscapes/audio_images/'
external_soundscapes_csv = pd.read_csv('../input/external-xenocanto-data-soundscapes/rich_train_metadata.csv')
external_soundscapes_csv['secondary_labels'] = external_soundscapes_csv['second_labels']
del external_soundscapes_csv['second_labels']
external_soundscapes_csv = external_soundscapes_csv.set_index("Unnamed: 0")

class DataConfig:
    # Stores Config vars pertaining to data
    soundscapes_val = False # Means to put all of the soundscapes(training) as your vlaidation set
    # IF False, validation becomes split between soundscapes(80/20 split) and monophone is added too.

print("Device:", DEVICE)

# Clean Up DF Functions

In [ ]:
def clean_df(df):
    # Cleans DataFrames from any erroneous entroes.
    to_drop = []
    for idx, row in tqdm(enumerate(df.iterrows())):
        row = row[1]
        if len(glob.glob(str(row.impath))) == 1:
            continue
        to_drop += [idx]
    df = df.reset_index()
    df = df.drop(to_drop)
    return df
        

In [ ]:
def get_df(mel_paths=MEL_PATHS, train_label_paths=TRAIN_LABEL_PATHS):
  df = None
  LABEL_IDS = {}
    
  for file_path in mel_paths:
    temp = pd.read_csv(str(file_path), index_col=0)
    temp["impath"] = temp.apply(lambda row: file_path.parent/"audio_images/{}/{}.npy".format(row.primary_label, row.filename), axis=1) 
    df = temp if df is None else df.append(temp)
    
    
  #df["secondary_labels"] = df["secondary_labels"].apply(literal_eval)

  for file_path in train_label_paths:
    with open(str(file_path)) as f:
      LABEL_IDS.update(json.load(f))
  LABEL_IDS['nocall'] = NUM_CLASSES - 1
  return LABEL_IDS, df
LABEL_IDS, df = get_df()

In [ ]:
def preprocess_external(df, base_path):
    # Preprocesses the External Dataset into the correct format.
    impath = []
    label_id = []
    secondary_label = []
    for row in tqdm(df.iterrows()):
        row = row[1]
        path = f'{base_path}{row.primary_label}/{row.filename}.npy'
        impath += [path]
        label_id += [LABEL_IDS[row.primary_label]]
    df['impath'] = impath
    df['label_id'] = label_id
    df['secondary_labels'] = [[]] * len(df)
    return df
def preprocess_soundscapes(df):
    # In order to model the test set, you need soundscapes.
    impath = []
    label_id = []
    site = []
    for row in df.iterrows():
        row = row[1]
        path = f'{external_soundscapes}{row.filename}.npy'
        if 'SSW' in path:
            site += ['SSW']
        else:
            site += ['COR']
        impath += [path]
        classes = row.primary_label.split()
        im_id = None
        classes_added = []
        for class_name in classes:
            try:
                
                if im_id is None:
                    if class_name == 'nocall':
                        im_id = '-1'
                        break
                    im_id = str(LABEL_IDS[class_name]) 
                    classes_added += [class_name]
                else:
                    if class_name not in classes_added:
                        im_id += f' {LABEL_IDS[class_name]}'
                        classes_added += [class_name]
            except:
                continue
        label_id += [im_id]
    df['impath'] = impath 
    df['label_id'] = label_id
    df['site'] = site
    # Segregate into External Soundscapes and Train Sound Scapes
    train_idx = []
    external_idx = []
    for idx, row in enumerate(df.iterrows()):
        row = row[1]
        filename = row.filename
        # Strip the base from the name
        base_filename = ''
        for i in range(len(filename) - 1, -1, -1):
            if filename[i] == '_':
                base_filename = filename[:i]
                break
        # Check if it exists
        num_files = len(glob.glob(f"{SOUNDSCAPES_PATH}{base_filename}*"))
        if num_files > 0:
            train_idx += [idx]
        else:
            external_idx += [idx]
    train = df.iloc[train_idx]
    external = df.iloc[external_idx] 
            
    return train, external

# Clean Up Dfs

In [ ]:
#BACKGROUND_CSV = preprocess_background(BACKGROUND_CSV)
external_soundscapes_csv, extra_soundscapes_csv = preprocess_soundscapes(external_soundscapes_csv)
#external_data_csv = preprocess_external(external_data_csv, external_data)
#external_data2_csv = preprocess_external(external_data2_csv, external_data2)
#external_data3_csv = preprocess_external(external_data3_csv, external_data3)


df = clean_df(df)
#BACKGROUND_CSV = clean_df(BACKGROUND_CSV)
external_soundscapes_csv =clean_df(external_soundscapes_csv)
extra_soundscapes_csv = clean_df(extra_soundscapes_csv)
#external_data_csv = clean_df(external_data_csv)
#external_data2_csv = clean_df(external_data2_csv)
#external_data3_csv = clean_df(external_data3_csv)



# Append Extra Data

In [ ]:
# Append Extra Data
#df = df.append(BACKGROUND_CSV)
#df = df.append(external_data_csv) # 40000 -> 100000
# Append External 2
#df = df.append(external_data2_csv) # 100000 -> 130000 # All Xeno-Canto Data Added.
#df = df.append(external_data3_csv)

# FOlds

In [ ]:
def stratified_KFold():
    # Special Splitting Strategy that splits the files from the soundspace separately(So they also have 4/5 split)
    # Performs a shuffled Stratified Split on the DataFrame.
    splitter = StratifiedKFold(shuffle = True, random_state = 42) 
    FOLDS = []
    SOUNDSCAPES_FOLDS = []
    if not DataConfig.soundscapes_val:
        for idx, (train, test) in enumerate(splitter.split(np.zeros(len(external_soundscapes_csv)), external_soundscapes_csv.site)):
            train_fold= df.append(external_soundscapes_csv.iloc[train]).append(extra_soundscapes_csv)
            test_fold = external_soundscapes_csv.iloc[test]
            FOLDS += [(train_fold, test_fold)]
            SOUNDSCAPES_FOLDS += [(external_soundscapes_csv.iloc[train].append(extra_soundscapes_csv), external_soundscapes_csv.iloc[test])]
    else:
        # No Folds, all monophone goes in train, all soundscapes in val
        FOLDS = [(df.append(extra_soundscapes_csv), external_soundscapes_csv)]
        SOUNDSCAPES_FOLDS += [(extra_soundscapes_csv, external_soundscapes_csv)]
    return FOLDS, SOUNDSCAPES_FOLDS

In [ ]:
FOLDS, SOUNDSCAPES_FOLDS = stratified_KFold()

# Now Append the External soundscapes Data
#if DataConfig.soundscapes_val:
df = df.append(external_soundscapes_csv).append(extra_soundscapes_csv)

# Model Definition

In [ ]:
class tmp_model(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.load_state_dict(torch.load(ResNestPath, map_location = DEVICE))
def load_prev_model(name):
    if "resnest" in name:
        model = resnest50_fast_1s1x64d()
    else:
        model = torchvision.models.resnext50_32x4d(pretrained=False)
    nb_ft = model.fc.in_features
    del model.fc
    num_cls =  264
    model.fc = nn.Linear(nb_ft, num_cls)
    model.load_state_dict(torch.load(ResNestPath, map_location = DEVICE))
    model.fc = nn.Identity()
    model.avgpool = nn.Identity()
    return model
class FeatureExtractor(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model_name = ModelConfig.model_name
        self.model = load_prev_model(self.model_name)
        
        self.conv1 = self.model.conv1
        self.bn1 = self.model.bn1
        self.act1 = self.model.relu
        self.maxpool = self.model.maxpool
        
        self.layer1 = self.model.layer1
        self.layer2 = self.model.layer2
        self.layer3 = self.model.layer3
        self.layer4 = self.model.layer4
        del self.model
    def forward(self, x):
        x = self.maxpool(self.bn1(self.act1(self.conv1(x))))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

# Transformer Blocks, Added on Top of CNN Features

In [ ]:
class SEDAttention(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # SED Attention, for Clipwise and Framewise Preds
        self.in_features = ModelConfig.feature_extractor_dim
        self.out_features = ModelConfig.num_classes
        
        self.framewise = nn.Conv1d(self.in_features, self.out_features, 1)
        self.attention = nn.Conv1d(self.in_features, self.out_features, 1)
    def forward(self, x):
        attention = F.softmax(self.attention(x), dim = -1) # (B, C, L)
        framewise = self.framewise(x) # (B, C, L)
        x = attention * framewise # (B, C, L)
        # Sum over time
        clipwise = x.sum(dim = -1) # (B, C)
        return clipwise
        
        
        
    
class SEDHead(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.feature_extractor = ModelConfig.feature_extractor_dim
        self.n_mels = 128
        self.dim_reduce = 32
        self.drop_prob = ModelConfig.dropout
        #self.Conv2d = nn.Conv2d(self.feature_extractor, self.feature_extractor, (self.n_mels // self.dim_reduce, 1))
        self.drop1 = nn.Dropout(self.drop_prob)
        self.drop2 = nn.Dropout(self.drop_prob)
        
        self.fc = nn.Linear(self.feature_extractor, self.feature_extractor) 
        self.relu = nn.ReLU(inplace = True)
        
        self.attention = SEDAttention()
    def forward(self, x):
        # X; Tensor(B, 2048, F, L):
        x = torch.mean(x, dim=2)  # BS x nb_ft x t

        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2  # BS x nb_ft x t
        x = self.drop1(x) 
        x = x.transpose(1, 2) # (B, L, 2048) 
        # --------FC-------
        x = self.relu(self.fc(x)) # (B, L, 2048)
        x = x.transpose(1, 2)
        x = self.drop2(x) # (B, 2048, L)
        # ---------Attention---------------
        return self.attention(x) # (B, C)

In [ ]:
class FullModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.feature_extractor = FeatureExtractor()
        self.head = SEDHead()
    def forward(self, x):
        features = self.feature_extractor(x)
        head = self.head(features)
        return head


In [ ]:
class ModelConfig:
    num_classes = NUM_CLASSES
    
    
    feature_extractor_dim = 2048
    transformer_dim = 768
    num_heads = 8
    
    model_name = 'resnest'
    dropout = 0.5
    num_layers = 2
    head = 'Conv2DAtt'
    num_channels = 3 # 1 for just Melspecs 
    
    use_mixup = False # Whether or not to use mixup augmentation.
    
    

# PreCache The Dataset

In [ ]:
def load_data(df):
    def load_row(row):
        # impath = TRAIN_IMAGES_ROOT/f"{row.primary_label}/{row.filename}.npy"
        image = np.load(str(row.impath))
        
        if len(image.shape) > 2:
            image = image[:MAX_READ_SAMPLES]
        else:
            image = np.expand_dims(image, axis = 0)
        return row.filename, image
    pool = joblib.Parallel(4)
    mapper = joblib.delayed(load_row)
    tasks = [mapper(row) for row in df.itertuples(False)]
    res = pool(tqdm(tasks))
    res = dict(res)
    return res

In [ ]:
# We cache the train set to reduce training time
#audio_image_store = None
if audio_image_store is None:
    audio_image_store = load_data(df)

# Dataset Definition

In [ ]:
def freq_mask(spec, F=30, num_masks=1, replace_with_zero=False):
    # Mel Spec Augments
    cloned = spec.clone()
    num_mel_channels = cloned.shape[0]
    
    for i in range(0, num_masks):        
        f = random.randrange(0, F)
        f_zero = random.randrange(0, num_mel_channels - f)

        # avoids randrange error if values are equal and range is empty
        if (f_zero == f_zero + f): return cloned

        mask_end = random.randrange(f_zero, f_zero + f) 
        if (replace_with_zero): cloned[f_zero:mask_end] = 0
        else: cloned[f_zero:mask_end] = cloned.mean()
    
    return cloned

#Export
def time_mask(spec, T=40, num_masks=1, replace_with_zero=False):
    cloned = spec.clone()
    len_spectro = cloned.shape[1]
    
    for i in range(0, num_masks):
        t = random.randrange(0, T)
        t_zero = random.randrange(0, len_spectro - t)

        # avoids randrange error if values are equal and range is empty
        if (t_zero == t_zero + t): return cloned

        mask_end = random.randrange(t_zero, t_zero + t)
        if (replace_with_zero): cloned[:,t_zero:mask_end] = 0
        else: cloned[:,t_zero:mask_end] = cloned.mean()
    return cloned
def pad_tensor(y):
    # Pads a Tensor to shape (128, 281) 
    shape = (128, 281)
    new_tensor = torch.zeros(shape)
    new_tensor[:, :y.shape[1]] = y
    return new_tensor
def lower_gain(y):
    # Lowers the Gain of the image
    lower_bound = 0.5
    upper_bound = 1
    gain = random.uniform(lower_bound, upper_bound)
    return y ** gain 
def mixup_with_val(y, idx):
    # Mixes up the melspec with the melspec from train_soundscapes(In order to produce "artificial" noise that exists in the test set.
    # This is only ever used at train, so idx is guaranteed to be 0
    soundscapes_df = SOUNDSCAPES_FOLDS[idx][0]
    soundscapes_df = soundscapes_df[soundscapes_df.primary_label == 'nocall']
    idx = random.randint(0, len(soundscapes_df) - 1)
    row = torch.tensor(np.load(soundscapes_df.iloc[idx].impath))
    background_mix = pad_tensor(row)
    
    mix_min = 0.4
    mix_max = 1 
    mix_quan = random.uniform(mix_min, mix_max)
    return y + mix_quan * background_mix
    
    
def cutmix_with_val(y, idx):
    soundscapes_df = SOUNDSCAPES_FOLDS[idx][0]
    soundscapes_df = soundscapes_df[soundscapes_df.primary_label == 'nocall']
    idx = random.randint(0, len(soundscapes_df) - 1) 
    row = torch.tensor(np.load(soundscapes_df.iloc[idx].impath))
    background_mix = pad_tensor(row)
    _, L = background_mix.shape
    start_idx = random.randint(0, L - 1)
    end_idx = random.randint(start_idx + 1, L)
    initial = y[:, :start_idx]
    end = y[:, end_idx:]
    middle= background_mix[:, start_idx:end_idx] / 255.0
    y = torch.cat([initial, middle, end], dim = -1)
    return y
#def cutout(y):
#    # Cutouts a Box in the Melspec, alternative to specAugment
    
def noise(y):
    
    return y + min(1,max(0.1, np.random.exponential(0.25))) 
def augment(y, idx):
    # Augmentation on MelSpecs
    
    if random.random() < 0.25:
        y = time_mask(y, num_masks = 2)
    if random.random() < 0.25:
        y = freq_mask(y, num_masks = 2)
    #if random.random() < 0.25:
    #    print("Noised")
    #    y = noise(y)
    #if random.random() < 0.5:
    #    y = mixup_with_val(y, idx)
    if random.random() < 0.5:
        y = lower_gain(y)
    return y # Augmentations aren't working right now.

In [ ]:
class BirdClefDataset(Dataset):

    def __init__(self, audio_image_store, idx, sr=SR, is_train=True, num_classes=NUM_CLASSES, duration=DURATION):
        
        self.idx = idx
        self.number = 0 if is_train else 1
        self.audio_image_store = audio_image_store
        self.meta = FOLDS[idx][self.number].copy().reset_index(drop=True)
        self.sr = sr
        self.is_train = is_train
        self.num_classes = num_classes
        self.duration = duration
        
        self.audio_length = self.duration*self.sr
        
        self.mean_train = np.array([0.485, 0.456, 0.406])
        self.mean_train =torch.tensor(np.expand_dims(np.expand_dims(self.mean_train, axis = -1), axis = -1))
        self.std_train = np.array([0.229, 0.224, 0.225])
        self.std_train = torch.tensor(np.expand_dims(np.expand_dims(self.std_train, axis = -1), axis = -1))
        self.stats = (self.mean_train, self.std_train) # ImageNet Stats 
        self.shape = (128, 801)
    
    
   

    def __len__(self):
        return len(self.meta)
    
    def __getitem__(self, idx):
        row = self.meta.iloc[idx]
        #secondary_labels = row.secondary_tensors
        image = self.audio_image_store[row.filename]
        image = torch.tensor(image[0]).to(torch.float32) / 255.0 # Sample From the Front.

        padded = torch.zeros(self.shape)
        padded[:, :image.shape[1]] = image
        image = padded
        if self.is_train:
            image = augment(image, self.idx)
        # One image is Regular power to db, another is cleaned powered power to db, one is pcen
        if ModelConfig.num_channels == 3:
            pcen = lb.pcen(image.numpy())
            mel = image
            power = image ** 1.5
            image = torch.stack([torch.tensor(pcen), torch.tensor(mel), torch.tensor(power)], dim = 0)
        else:
            image = torch.stack([image, image, image])
       
        
        t = np.zeros(self.num_classes, dtype=np.float32) 
        label_id = row.label_id
        try:
            label_id = label_id.item()
        except:
            pass
        
        if type(label_id) == type(1.0) or type(label_id) == type(1):
            t[row.label_id] = 1
        else:
            
            # Soundscape, so multiple things
            labels = row.label_id.split()
            for label in labels:
                if label == '-1':
                    t = np.zeros(self.num_classes, dtype = np.float32)
                    break
                t[int(label)] = 1
                
        
        return torch.tensor(image).to(torch.float32), torch.tensor(t).to(torch.float32)

In [ ]:
dataset = BirdClefDataset(audio_image_store, 0, is_train = True)

In [ ]:
count = 0
for images, labels in dataset:
    lbd.specshow(images[1].numpy())
    plt.show()
    count += 1
    if count == 32:
        break

In [ ]:
def label_smooth(primary): 
    primary = primary.clone() 
    ones = primary == 1 
    not_ones = primary == 0 
    primary[ones] = 0.995 
    primary[not_ones] = 0.0025 
    return primary

# mixup and cutmix.

In [ ]:
def mixup(x, y):
    # Performs Mixup on Melspecs
    # x: Tensor(B, 128, 281)
    # Mixup sampled from beta distribution.
    beta = 0.4
    gamma = random.beta(beta, beta)
    gamma = max(1-gamma, gamma)
    shuffle = torch.randperm(x.shape[0]).to(x.device)
    x = gamma*x + (1-gamma)*x[shuffle]
    y = gamma*y + (1-gamma)*y[shuffle]
    # for hard mixup, anything that isn't 0 is set to 1.
    return x, y # efficient mixup impletation(not completely random, but it should be fine.)
    
def cutmix(batch, y):
    # Performs Time wise Cutmix on batch of melspecs
    pass

# collate functions

In [ ]:
def train_collate(values):
    images = torch.stack([value[0] for value in values])
    labels = torch.stack([value[1] for value in values]) # (B, 128, 281)
    # ----------MixUp--------(or Cutmix?)
    if ModelConfig.use_mixup:
        images, labels = mixup(images, labels)
    return images, labels
def val_collate(values):
    images = torch.stack([value[0] for value in values])
    labels = torch.stack([value[1] for value in values])
    return images, labels 
    

# Training the model

In [ ]:
CRITERION = nn.BCEWithLogitsLoss(reduction = 'mean') # TODO: Mask Secondary Outputs Loss.
def loss_fn(y_pred, primary):
    # Primary: Tensor(B)
    # Secondary: Tensor(B, C)
    # Y_Pred: Tensor(B, C)
    # TODO Split the Losses.
    B, C = primary.shape
    smooth = label_smooth(primary)
    y_pred = y_pred.float()
    loss = CRITERION(y_pred, smooth)

    return loss

# Metrics

In [ ]:
class F1_score(Metric):
    def __init__(self):
        self.thresholds = np.arange(0.0, 1.0, 0.01)
        self.f1_scores = [0.0 for i in range(len(self.thresholds))]
        self.count = 0
    def round_pred(self, y_pred, thresh):
        ones = y_pred >= thresh
        logits = torch.zeros_like(y_pred, device = y_pred.device)
        logits = logits + ones.int()
        return logits
    def round_true(self, y_true):
        ones = y_true >= 0.5
        logits = torch.zeros_like(y_true, device = y_true.device)
        logits = logits + ones.int()
        return logits
    def metric(self, y_pred, target, thresh_idx):
        # Given y_pred = Tensor(B, C) and primary = Tensor(B, C), Computes the Row-wise F1 Score
        # Threshold Predictions
        predictions = self.round_pred(y_pred, self.thresholds[thresh_idx])
        # PRedictions: (B, C)
        zeros = torch.sum(predictions, axis = -1) == 0
        predictions[zeros, NUM_CLASSES - 1] = 1 # No Call
        tp = (predictions * target).sum(1)
        fp = (predictions * (1 - target)).sum(1)
        fn = ((1 - predictions) * target).sum(1)
        
        eps = 1e-9
        f1 = (tp + eps) / (tp + (fp + fn) / 2 + eps)
        # Accumulate the f1 score
        self.f1_scores[thresh_idx] += f1.mean().item()
    def compute_f1_score(self, y_pred, primary):
        primary = self.round_true(primary)
        for th_idx in range(len(self.thresholds)):
            y_p = self.round_pred(y_pred, self.thresholds[th_idx])
            self.metric(y_p, primary, th_idx)
        self.count += 1
    def accumulate(self, learn):
        # y_pred: Tensor(B, C)
        y_pred = torch.sigmoid(learn.pred).cpu()
        primary = learn.y.cpu()
        self.compute_f1_score(y_pred, primary)
        
    def reset(self):
        self.f1_scores = [0.0 for i in range(len(self.f1_scores))]
        self.count = 0
        
    @property
    def value(self):
        eps = 1e-8
        # Computes the best threshold and result
        best = 0
        best_th = 0
        for th_idx in range(len(self.f1_scores)):
            if self.f1_scores[th_idx] > best:
                best = self.f1_scores[th_idx]
                best_th = self.thresholds[th_idx]
        best = (best + eps) / (self.count + eps)
        print(f"F1Score: {best}")
        return best
class F1Score_th(Metric):
    def __init__(self):
        self.thresholds = np.arange(0.0, 1.0, 0.01)
        self.f1_scores = [0.0 for i in range(len(self.thresholds))]
        self.count = 0
    def round_pred(self, y_pred, thresh):
        ones = y_pred >= thresh
        logits = torch.zeros_like(y_pred, device = y_pred.device)
        logits = logits + ones.int()
        return logits
    def round_true(self, y_true):
        ones = y_true >= 0.5
        logits = torch.zeros_like(y_true, device = y_true.device)
        logits = logits + ones.int()
        return logits
    def metric(self, y_pred, target, thresh_idx):
        # Given y_pred = Tensor(B, C) and primary = Tensor(B, C), Computes the Row-wise F1 Score
        # Threshold Predictions
        predictions = self.round_pred(y_pred, self.thresholds[thresh_idx])
    
        tp = (predictions * target).sum(1)
        fp = (predictions * (1 - target)).sum(1)
        fn = ((1 - predictions) * target).sum(1)
        
        eps = 1e-9
        f1 = (tp + eps) / (tp + (fp + fn) / 2 + eps)
        # Accumulate the f1 score
        self.f1_scores[thresh_idx] += f1.mean().item()
    def compute_f1_score(self, y_pred, primary):
        primary = self.round_true(primary)
        for th_idx in range(len(self.thresholds)):
            y_p = self.round_pred(y_pred, self.thresholds[th_idx])
            self.metric(y_p, primary, th_idx)
        self.count += 1
    def accumulate(self, learn):
        # y_pred: Tensor(B, C)
        y_pred = torch.sigmoid(learn.pred).cpu()
        primary = learn.y.cpu()
        self.compute_f1_score(y_pred, primary)
        
    def reset(self):
        self.f1_scores = [0.0 for i in range(len(self.f1_scores))]
        self.count = 0
        
    @property
    def value(self):
        eps = 1e-8
        # Computes the best threshold and result
        best = 0
        best_th = 0
        for th_idx in range(len(self.f1_scores)):
            if self.f1_scores[th_idx] > best:
                best = self.f1_scores[th_idx]
                best_th = self.thresholds[th_idx]
        best = (best + eps) / (self.count + eps)
        return best_th
        
    
class Accuracy(Metric):
    def __init__(self):
        self.accuracy = 0
        self.count = 0
    def round_pred(self, y_pred):
        y_pred = torch.sigmoid(y_pred)
        ones = y_pred >= 0.5
        scores = torch.zeros_like(y_pred, device = y_pred.device)
        scores[ones] = 1
        return scores
    def round_true(self, y_true):
        ones = y_true >= 0.5
        logits = torch.zeros_like(ones, device = ones.device)
        logits = logits + ones.int()
        return logits
    def accumulate(self,learn):
        y_pred = self.round_pred(learn.pred)
        primary = self.round_true(learn.y)
        # y_pred: Tensor(B, C)
        # primary: Tensor(B, C)
        B, C = y_pred.shape
        tp = torch.sum(y_pred == primary)
        acc = tp / (B * C)
        
        self.accuracy += acc.item()
        self.count += 1
    def reset(self):
        self.accuracy = 0 
        self.count = 0
    @property
    def value(self):
        eps = 1e-8
        return round((self.accuracy + eps) / (self.count + eps), 3)

# TRaining Config

In [ ]:
class TrainingConfig:
    start_lr = 1e-3
    max_lr = 5e-3
    min_lr = 1e-4
    warm_steps = 0.1
    peak_steps = 0.2 # 0.3 ramp up, 0.7 rabsamp down = super-convergence
    
    num_epochs = 9
    train_steps = len(FOLDS[0][0])
    
    total_steps = num_epochs * train_steps
    
    weight_decay = 0
    

# Training Fn

# DataLoader Config

In [ ]:
class DataLoaderConfig:
    def __init__(self, is_train):
        num_workers = 4
        pin_memory = True
        bs = 32
        create_batch = train_collate if is_train else val_collate
        shuffle = is_train
        self.config = {'num_workers': num_workers, 'pin_memory': pin_memory, 'bs': bs, 'shuffle': shuffle, 'create_batch': create_batch}
        


In [ ]:
def train_folds_fast_ai(folds, load_prev = False):
    # Fast Ai, with their variety of tricks, trains faster and better
    for fold_idx in folds:
        # Overfit Testing
        train = BirdClefDataset(audio_image_store, fold_idx, is_train = True)
        val = BirdClefDataset(audio_image_store,fold_idx, is_train = False)#BirdClefDataset(audio_image_store, val, is_train = False)
        
        train_config = DataLoaderConfig(is_train = True)
        val_config = DataLoaderConfig(is_train = False)
        
        train = DataLoader(train, **train_config.config)
        val = DataLoader(val, **val_config.config)
        dls = DataLoaders(train, val)
        model = FullModel()
        if torch.cuda.is_available(): dls.cuda(),model.cuda()
        learn = Learner(dls, model, loss_func=loss_fn,
                metrics=[F1Score_th(), F1_score], opt_func = Adam, lr = TrainingConfig.start_lr, wd = TrainingConfig.weight_decay).to_fp16()
        cbs = [
            SaveModelCallback(monitor = 'f1_score',comp = np.greater, fname = f'model_{fold_idx}'),
            EarlyStoppingCallback(monitor = 'f1_score', comp = np.greater, patience = 5),
            ReduceLROnPlateau(monitor = 'f1_score', comp = np.greater, patience = 3, min_lr = 1e-7)
        ]
        if load_prev:
            learn.load(f'final_{fold_idx}')
        learn.fit_one_cycle(TrainingConfig.num_epochs, lr_max = TrainingConfig.max_lr, cbs = cbs, wd = TrainingConfig.weight_decay)
        learn.save(f"final_{fold_idx}")

In [ ]:
!cp -r ../input/sed1s-bird-clef/models/ ./

In [ ]:
FOLDS_IDX = [2]
model = train_folds_fast_ai(FOLDS_IDX, load_prev = True)